In [1]:
# データ拡張テスト
# http://aidiary.hatenablog.com/entry/20161212/1481549365
# https://keras.io/preprocessing/image/

from keras.layers import Input, Dense, Flatten
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
import sys, glob, math, os
import cv2
import numpy as np

Using TensorFlow backend.


In [2]:
# image sizes
img_rows, img_cols = 224, 224

In [3]:
def load_imgs(dir,normalize=0):
    # load image data from dir

    data = []
    fname = []
    path  = dir + '/*.png'
    files = sorted(glob.glob(path))

    for fl in files:
        flbase = os.path.basename(fl)
        print('reading %s'%format(fl))

        img = cv2.imread(fl)
        img = cv2.resize(img, (img_rows,img_cols))
        img = np.array(img, dtype=np.float)
        if normalize == 1:
            img -= np.mean(img)
            img /= np.std(img)

        data.append(img)
        fname.append(fl)

    data = np.array(data,dtype=np.float)

    return data, fname
# end of load_imgs

In [4]:
#imgs, name = load_imgs('./dataset/valid/image')
#imgs2, name2 = load_imgs('./dataset/valid/gt')
#dst_dir = './dataset/valid/image-aug3'
#dst_dir2 = './dataset/valid/gt-aug3'
imgs, name = load_imgs('./dataset/train/image')
imgs2, name2 = load_imgs('./dataset/train/gt')
dst_dir = './dataset/train/image-aug3'
dst_dir2 = './dataset/train/gt-aug3'
os.mkdir(dst_dir)
os.mkdir(dst_dir2)

reading ./dataset/train/image/fujiwara0218.png
reading ./dataset/train/image/fujiwara0285.png
reading ./dataset/train/image/fujiwara0325.png
reading ./dataset/train/image/fujiwara0381.png
reading ./dataset/train/image/fujiwara0469.png
reading ./dataset/train/image/fujiwara0509.png
reading ./dataset/train/image/fujiwara0565.png
reading ./dataset/train/image/fujiwara0621.png
reading ./dataset/train/image/fujiwara0702.png
reading ./dataset/train/image/fujiwara0757.png
reading ./dataset/train/image/fujiwara0845.png
reading ./dataset/train/image/fujiwara0892.png
reading ./dataset/train/image/higuchi0214.png
reading ./dataset/train/image/higuchi0267.png
reading ./dataset/train/image/higuchi0334.png
reading ./dataset/train/image/higuchi0386.png
reading ./dataset/train/image/higuchi0447.png
reading ./dataset/train/image/higuchi0519.png
reading ./dataset/train/image/higuchi0679.png
reading ./dataset/train/image/higuchi0727.png
reading ./dataset/train/image/higuchi0762.png
reading ./dataset/trai

FileExistsError: [Errno 17] File exists: './dataset/train/image-aug3'

In [5]:
data_gen_args = dict(featurewise_center=False,
                     featurewise_std_normalization=False,
                     rotation_range=90.,
                     width_shift_range=0.2,
                     height_shift_range=0.2,
                     zoom_range=0.3,
                     channel_shift_range=20.0,
                     cval=0.,
                     horizontal_flip=True,
                     vertical_flip=True )

#data_gen_args = dict(rotation_range=90.)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(imgs, augment=True)
mask_datagen.fit(imgs2, augment=True)

g  = image_datagen.flow(imgs, batch_size=1, seed=1)
g2 = mask_datagen.flow(imgs2, batch_size=1, seed=1)

n = 0

for i in range(2000):
    x = g.next()
    x2 = g2.next()
    cv2.imwrite('%s/%05d.jpg'%(dst_dir,n),x[0].astype(np.uint8))
    cv2.imwrite('%s/%05d.jpg'%(dst_dir2,n),x2[0].astype(np.uint8))
    n = n + 1
    
cv2.destroyAllWindows()